In [1]:
# Python
# Third
# Property
from ls_lib import *
from etl_b3_stocks import create_long_short_dataset
pd.set_option('display.max_rows', 150)


### TODO
- Type hint
- Field
- Optional
- Document functions
- Pytest/Unit Tests (Udacity content)
- log (Udacity content)
- Web-App (Udacity 6.17 em diante[Plotly em diante])

In [29]:
create_long_short_dataset(start_date="2018-01-01")

## L&S Screaning

Gerênciamento de risco
- 160 períodos: 30% do Capital para cada trade.
- 200 períodos: 50% do Capital para cada trade.
- 252 períodos: 100% do Capital para cada trade.


Teste ADF
- 1% -4,32%
- 5% -3,78%
- 10% -3,50%

Interpretação do Beta
- Para cada 1 real do ativo 1, operar beta*reais do ativo 2

Entre A/B e B/A, escolher o par com menor ADF

In [18]:
df_stocks = load_long_short_dataset(max_date = "2024-08-26")

In [19]:
# Filtro dataset

lst_pairs = getPairs(df_stocks)
periods = 200
date_col = 'date'

args = [{"pair":pair, "dataset": df_stocks, "periods": periods, "date_col": date_col} for pair in lst_pairs]

df_stats = getPairsStats(args, workers = 8)
df_stats["periodo"] = periods

In [20]:
# Screaning
df_screaning = df_stats[
    (df_stats['adf'] <= -4.32) & 
    (abs(df_stats['desv']) >= 1.90) & 
    (abs(df_stats['desv']) < 2.2) & 
    (df_stats['beta'] >= 0.75) & 
    (df_stats['beta'] < 1.25) & 
    (df_stats['halflife'] <= 11)]

df_screaning = double_cointegration(df_screaning, check = True).sort_values(by='adf')

df_screaning.reset_index(drop=True)

,pair,adf,beta,desv,op_type,halflife,p_value,periodo


In [21]:
for n_pair in range(len(df_screaning)):
    print(n_pair)
    print(df_screaning.iloc[n_pair,0])
    print("Beta: ", df_screaning.iloc[n_pair,2])
    print("Desv: ", df_screaning.iloc[n_pair,3])
    print("Halflife: ", df_screaning.iloc[n_pair,4])

    # Residual
    asset1 = df_screaning.iloc[n_pair,0].split('/')[0]
    asset2 = df_screaning.iloc[n_pair,0].split('/')[1]
    df_res = df_stocks[[date_col, asset1, asset2]].sort_values(by=date_col, ascending=False)[:periods]
    X = df_res.iloc[:, 1].values.reshape(-1, 1)
    Y = df_res.iloc[:, 2].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)

    Y_pred = linear_regressor.predict(X)
    df_res[asset1+'/'+asset2] = Y - Y_pred  # Residuals

    df_res = df_res[[date_col, asset1+'/'+asset2]]
    
    df_res_aux = normalize_column(df_res, asset1+'/'+asset2)

    plt.plot(df_res_aux)
    plt.show()

    # Regression
    plt.scatter(X, Y)
    plt.plot(X, Y_pred, color='red')
    plt.show()